In [4]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
ret,frame = cap.read()

#Detecting Face and setting it as a starting point
face_cascade = cv2.CascadeClassifier('../DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rects = face_cascade.detectMultiScale(frame)

#Convert numpy array to tuple
(face_x,face_y,w,h) = tuple(face_rects[0]) 
track_win = (face_x,face_y,w,h)

#Creating a region of interest in hsv color space
roi = frame[face_y:face_y+h,face_x:face_x+w]
roi_hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

#Calculating histogram and normalizing the limits as 0 to 255
roi_hist= cv2.calcHist([roi_hsv],[0],None,[180],[0,180])
roi_hist = cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

#Setting Termination Criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1)

while True:
    ret,frame = cap.read()
    if ret == True:
        frame_hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        
        # Calculate the Back Projection based off the roi_hist created earlier
        
        dst = cv2.calcBackProject([frame_hsv],[0],roi_hist,[0,180],1)
        #Tracking the window in the direction of weighted mean with CamShift vector
        ret,track_win = cv2.CamShift(dst,track_win,term_crit)
        
        #ret return rotated rectangle of 
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img = cv2.polylines(frame,[pts],True,(255,255,255),4)
        cv2.imshow('Face Detection',img)
        
        k = cv2.waitKey(1) & 0xff
        if k == 27:
            break
        
    else:
        break

cv2.destroyAllWindows()
cap.release()